In [95]:
import json,csv
import pandas as pd

# d = pd.read_json('data.json')
# print(d)

# 1. Parse the JSON into a Python dictionary.
with open('data.json','r') as f:
    data = json.load(f)
    print(data)


{'stores': [{'name': 'Store 1', 'fruits': {'apple': {'initial_stock': 50, 'stock': 10, 'cost_price': 20, 'selling_price': 40}, 'banana': {'initial_stock': 100, 'stock': 15, 'cost_price': 5, 'selling_price': 10}}, 'groceries': {'rice': {'initial_stock': 200, 'stock': 30, 'cost_price': 30, 'selling_price': 50}, 'wheat': {'initial_stock': 150, 'stock': 25, 'cost_price': 25, 'selling_price': 45}}, 'vegetables': {'carrot': {'initial_stock': 80, 'stock': 8, 'cost_price': 10, 'selling_price': 20}, 'potato': {'initial_stock': 100, 'stock': 40, 'cost_price': 12, 'selling_price': 25}}, 'dairy': {'milk': {'initial_stock': 60, 'stock': 5, 'cost_price': 20, 'selling_price': 30}, 'cheese': {'initial_stock': 30, 'stock': 12, 'cost_price': 50, 'selling_price': 80}}}, {'name': 'Store 2', 'fruits': {'apple': {'initial_stock': 40, 'stock': 9, 'cost_price': 20, 'selling_price': 40}, 'banana': {'initial_stock': 120, 'stock': 12, 'cost_price': 5, 'selling_price': 10}}, 'groceries': {'rice': {'initial_stock'

In [96]:
# to dataframe

# Flatten the nested dictionary structure
flattened_data = []
for store in data['stores']:
    store_name = store['name']
    for category, items in store.items():
        if category == 'name':
            continue
        for item_name, item_data in items.items():
            flattened_data.append( {
                'Store': store_name,
                'Category': category,
                'Item': item_name,
                'Initial Stock': item_data['initial_stock'],
                'Current Stock': item_data['stock'],
                'Cost Price': item_data['cost_price'],
                'Selling Price': item_data['selling_price']
            })
            

# Create DataFrame
df = pd.DataFrame(flattened_data)
print(df)

       Store    Category    Item  Initial Stock  Current Stock  Cost Price  \
0    Store 1      fruits   apple             50             10          20   
1    Store 1      fruits  banana            100             15           5   
2    Store 1   groceries    rice            200             30          30   
3    Store 1   groceries   wheat            150             25          25   
4    Store 1  vegetables  carrot             80              8          10   
..       ...         ...     ...            ...            ...         ...   
75  Store 10   groceries   wheat            139             21          25   
76  Store 10  vegetables  carrot             86              4          10   
77  Store 10  vegetables  potato             97             27          12   
78  Store 10       dairy    milk             61              6          20   
79  Store 10       dairy  cheese             29             14          50   

    Selling Price  
0              40  
1              10  
2  

In [94]:
df.reset_index(inplace=True)

In [97]:
# 2. Compute the following for each category (across all stores):

#total units sold
df['Units Sold'] = df['Initial Stock'] - df['Current Stock']

#total profit
df['Profit'] = (df['Selling Price'] - df['Cost Price']) * df['Units Sold']

#
#filtering
df.rename(columns={'Store' : 'Store Name', 'Current Stock' : 'Closing Stock'} , inplace = True)
df['Warning'] = ["YES" if i < 10 else " " for i in df['Closing Stock']]

df[['Store Name','Category', 'Item' ,'Initial Stock' ,'Closing Stock' , 'Units Sold' , 'Profit','Warning']]

,Store Name,Category,Item,Initial Stock,Closing Stock,Units Sold,Profit,Warning
0,Store 1,fruits,apple,50,10,40,800,
1,Store 1,fruits,banana,100,15,85,425,
2,Store 1,groceries,rice,200,30,170,3400,
3,Store 1,groceries,wheat,150,25,125,2500,
4,Store 1,vegetables,carrot,80,8,72,720,YES
...,...,...,...,...,...,...,...,...
75,Store 10,groceries,wheat,139,21,118,2360,
76,Store 10,vegetables,carrot,86,4,82,820,YES
77,Store 10,vegetables,potato,97,27,70,910,
78,Store 10,dairy,milk,61,6,55,550,YES


In [98]:
# Generate a CSV Report (report.csv)
report = df.to_csv('report.csv',index=False)

In [105]:
#details for summary
summary = df.groupby('Category',as_index = False)[['Units Sold' , 'Profit']].sum()
summary


,Category,Units Sold,Profit
0,dairy,755,11270
1,fruits,1349,13525
2,groceries,2984,59680
3,vegetables,1444,16543


In [ ]:
# Display the nightly summary report in the terminal



print("Nightly Sales Summary")
print(f"-"*40)
print(f"{'Category':<13} | {'Units Sold' :^10} | {'Profit' :>10}")


for  _,row in summary.iterrows():

    category = row['Category']
    units_sold = row['Units Sold']
    profit = row['Profit']
    print(f"{category:<13} | {units_sold:>10} | ₹{profit :>9,}")

print("-" * 40)
total_profit = summary['Profit'].sum()
print(f"{'TOTAL PROFIT':>27} ₹{total_profit:>9,}")


print("Low Stock Warning")
print(f"-"*40)

for _, row in df.iterrows():
    if row['Warning'] == 'YES':
        store_name = row['Store Name']
        item = row['Item']
        remaining = row['Closing Stock']
        print(f"{store_name} - {item} - Remaining: {remaining}")


Nightly Sales Summary
----------------------------------------
Category      | Units Sold  |     Profit
dairy         |        755 | ₹   11,270
fruits        |       1349 | ₹   13,525
groceries     |       2984 | ₹   59,680
vegetables    |       1444 | ₹   16,543
----------------------------------------
               TOTAL PROFIT ₹  101,018
Low Stock Warning
----------------------------------------
Store 1 - carrot - Remaining: 8
Store 1 - milk - Remaining: 5
Store 2 - apple - Remaining: 9
Store 2 - milk - Remaining: 3
Store 3 - apple - Remaining: 7
Store 3 - carrot - Remaining: 9
Store 3 - milk - Remaining: 6
Store 4 - apple - Remaining: 8
Store 4 - carrot - Remaining: 5
Store 4 - milk - Remaining: 2
Store 5 - carrot - Remaining: 6
Store 5 - milk - Remaining: 4
Store 6 - apple - Remaining: 6
Store 6 - carrot - Remaining: 7
Store 6 - milk - Remaining: 5
Store 7 - carrot - Remaining: 6
Store 7 - milk - Remaining: 3
Store 8 - carrot - Remaining: 5
Store 8 - milk - Remaining: 6
Store 9 -